In [141]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import datetime 
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.messages import HumanMessage

load_dotenv()

True

### Define the schema

In [142]:
from typing import  List
from pydantic import BaseModel, Field

class Reflection(BaseModel):
     missing: str = Field(description="Critique of what is missing.")
     superfluous: str = Field(description="Critique of what is superfluous.")

class AnswerQuestion(BaseModel):
     """ Answer to the Question"""

     answer: str = Field(description="~250 word detailed answer to the question")
     search_queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer")
     reflect: Reflection = Field(description="Your refelcton on the initial answer")

class ReviseAnswer(AnswerQuestion):
     """Revise your original answer to your question."""
     references: List[str] = Field(description="Citations motivating your update answer.")


## define the prompt template

In [143]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
         """ You are expert AI Researcher
         Current Time: {time}
         
         1. {first_instruction}
         2. Reflect and critique your answer. Be severe to maximize improvement.
         3. After the reflectiom, **List 1-3 search queires seperately** for researching improvements.
         Do not include them inside the reflection
         """),
         MessagesPlaceholder(variable_name="messages"),
         ("system",
          "Answer the user's question above using thre required format.")
    ]
).partial(  # .partials is used to prepopulate prompt template before invoking it on LLM
    time=lambda: datetime.datetime.now().isoformat()
)

### build the full prompt template, the llm and the parser

In [144]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250word answer"
) # the full prompt template

tools=[AnswerQuestion]  #build the tools
llm = ChatOpenAI(model="gpt-4o")  # build the llm
pydantic_parser = PydanticToolsParser(tools=tools) # the parser

### Create the first responder chain

In [145]:
first_responder_chain = first_responder_prompt_template | llm.bind_tools(tools=tools,tool_choice='AnswerQuestion') 


### prompt instruction for the revision and it chain

In [146]:
revise_instructions = """
Revise your  previous answer using the new information
- You should use the revious critique to add important information to your answer.
- You MUST inlude numerical citations in your revised answer to ensure it can ver verified
- add a "References" section to the botton of your anwser (which does not count towards the word limit). 
in form of:
    - [1] https://example.com
    - [2] https://example.com
- You should use the previous critique to remove superfluous information from your answer and make SURE it is not more th
250 words.
"""

# add the actor template and the revisor template together
 # add the actor template and the revisor template together
revisor_chain = actor_prompt_template.partial(
    first_instruction = revise_instructions
) | llm.bind_tools(tools=[ReviseAnswer], tool_choice='ReviseAnswer')

In [147]:
import json
from typing import List, Dict, Any 
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage
from langchain_community.tools import TavilySearchResults
# create the Tavily Search tool
tavily_tools = TavilySearchResults(max_results=3)



def execute_tools(state:List[BaseMessage]) ->List[BaseMessage]:
    #from the list of message, get the last message (which is usally an AIMessage tool message)
    last_ai_message = state[-1] 

    #extract tool calls from the AI Message
    # start by checking if the last ai message is a tool call, if not return []
    if not hasattr(last_ai_message,"tool_calls") or not last_ai_message.tool_calls:
        return []
    
    tool_messages = [] #this will store all the tools
    # if last_ai_message is a tool then loop through all the available tools
    for tool_call in last_ai_message.tool_calls:
        if tool_call["name"] in ["AnswerQuestion", "ReviseAnswer"]: # is there a tool called AnswerQuestion or ReviseAnswer
            call_id = tool_call["id"] #if you found any of then the get their id
            search_queries = tool_call["args"].get("search_queries", []) # also get the search_queries list data inside args dict

            #loop through the list in search_queries and store the result in query_results
            #each search_queries item contain a query string. This will be passed to tavily to generate respond
            query_results = {}
            for query in search_queries:
                result = tavily_tools.invoke(query) # this uses tavily tool to generate a list of 3 response for each query in search_queries
                query_results[query]= result # store the response in the query_results dictionary

            #create a tool with the generated result
            tool_messages.append(
                ToolMessage(
                    tool_call_id = call_id,
                    content = json.dumps(query_results),
                )
            )
    return  state + tool_messages

## RUN AND DEBUG CODE

In [148]:
# INITIALIZE THE FIRST MESSAGE
initial_message = HumanMessage(content="Write a blog post on how small businesses can leverage AI to grow")

# CREATE THE FIRST STATE WITH THE HumanMessage
state = [initial_message]  # this returns a list of HumanMessage (at this point it is only one)
state

[HumanMessage(content='Write a blog post on how small businesses can leverage AI to grow', additional_kwargs={}, response_metadata={})]

### pass the state into the first_responder_chain - this responds with an AIMessage output

In [149]:
## PASS THE STATE INTO THE DRAFT NODE -first_responder_chain
first_responder_output = first_responder_chain.invoke({"messages": state})
 

### add the AIMessage response from first_responder to the state
state now contains initial message + first_responder response (which is an AIMessage)

In [150]:
state.append(first_responder_output)
 

### execute the execute_tools to build the ToolMessage

In [151]:
tool_outputs = execute_tools(state)


In [153]:

## update the state to include tool_output (execute_tool alread added state+its_result 
state = tool_outputs

In [154]:
len(state)

3

### Run the Revisor Chain adding the state

In [155]:
revisor_output = revisor_chain.invoke({"messages": state})

In [157]:
state.append(revisor_output)

In [162]:
len(state)

4

###  Loop Back to execute_tools (if applicable) or end

In [163]:
### Run one more executor and add the result to the state
tool_outputs = execute_tools(state)
state=tool_outputs
## check the last state result

len(state)

5

### loop through executor and revisor then print out the last record

In [164]:
last_output = state[-1]

In [165]:
last_output

ToolMessage(content='{"Specific AI tools for small businesses": [{"title": "AI for small business owners: Tools, benefits, and use cases", "url": "https://printify.com/blog/ai-for-small-business/", "content": "AI is accessible and practical for small businesses, helping automate assignments, improve decision-making, and reduce costs.\\n   Businesses of all sizes can use AI tools like Jasper, Tidio, and Zapier to streamline workflows, answer customer questions, and maximize productivity.\\n   Generative AI helps create content quickly, from product descriptions to ad creatives. It supports growth without needing extra staff or time. [...] Start with small wins. Use AI to automate tasks like email responses, social media posts, or customer onboarding. Tools like Zapier, Notion AI, or Tidio allow small businesses to set up workflows that save time and increase efficiency.\\n\\nOver time, you can build more complex automations across your business operations.\\n\\n How much does AI cost fo

In [ ]:
tool_outputs

In [77]:
## display the 
#get the queries in the 
# queries = state[-1].tool_calls[0]['args']['search_queries']

# query_results = {}
# for query in queries:
#     result = tavily_tools.invoke(query)
#     query_results[query]= result
# from IPython.display import display, Markdown

# for topic, description in query_results.items():
#     display(Markdown(f"### {topic}"))  # Bold in Markdown
#     # print(description)
#     for msg in description:
#         # print(msg)
#         display(Markdown(f"**{msg['title']}**"))
#         print(msg['url'])
#         display(Markdown(f"{msg['content']}"))
   
#     display(Markdown(f"___"))


### Run the Execute_tool to know if to end the code or run a revisor

In [109]:
state.append(first_responder_output)
len(state)

2

In [108]:
first_responder_output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X4uucKLeZwe1wr8964H47lQ6', 'function': {'arguments': '{"answer":"In today’s rapidly evolving digital landscape, small businesses face unique challenges, yet opportunities abound through the strategic use of artificial intelligence (AI). While AI might seem daunting for smaller enterprises due to perceived costs or complexity, it is, in fact, a powerful ally in optimizing operations, enhancing customer experience, and driving growth. \\n\\nFirstly, AI can automate routine tasks, freeing up time for business owners and employees to focus on more strategic activities. Tools like chatbots can handle customer inquiries around the clock, improving service levels while reducing the need for extensive human resources. This is particularly beneficial for businesses that operate online, providing immediate responses that help maintain customer engagement.\\n\\nAI also offers unparalleled insights through data analytics. By anal

In [117]:
state[1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X4uucKLeZwe1wr8964H47lQ6', 'function': {'arguments': '{"answer":"In today’s rapidly evolving digital landscape, small businesses face unique challenges, yet opportunities abound through the strategic use of artificial intelligence (AI). While AI might seem daunting for smaller enterprises due to perceived costs or complexity, it is, in fact, a powerful ally in optimizing operations, enhancing customer experience, and driving growth. \\n\\nFirstly, AI can automate routine tasks, freeing up time for business owners and employees to focus on more strategic activities. Tools like chatbots can handle customer inquiries around the clock, improving service levels while reducing the need for extensive human resources. This is particularly beneficial for businesses that operate online, providing immediate responses that help maintain customer engagement.\\n\\nAI also offers unparalleled insights through data analytics. By anal